# Identifying devices with ChatGPT

In [ ]:
import importlib
import pandas as pd
import requests
import json
import functools
import glob
import base64
import multiprocessing.pool
import tldextract
import tqdm
import matplotlib
import threading
import itertools
import requests
import pickle
import whois

# Common functions

In [ ]:
import openai

OPENAI_API_KEY = 'sk-PLACE-YOUR-KEY-HERE'
openai.api_key = OPENAI_API_KEY
models = openai.Model.list()


def chat_completion(messages, max_tokens=20):
    return openai.Completion.create(
        engine='text-davinci-003',
        prompt= messages,
        max_tokens=max_tokens,
        temperature=0,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )

def ask_gpt_raw(messages, max_tokens=20):
    response = chat_completion(messages, max_tokens)
    return response.choices[0].text.translate(str.maketrans('', '', '\n\r\t'))

In [ ]:
V2_DIR = 'DEVICE FILE'
V3_DIR = 'DEVICE FILE'
TMP_DIR = 'DEVICE FILE'

T_LOCK = threading.Lock()

In [ ]:
ask_gpt_raw('What is the capital of Tibet?')

# Preprocess raw data

## Load device info

In [ ]:
df_list = []

for filename in glob.glob(V2_DIR + 'devices.*'):
    df = pd.read_csv(filename)
    df_list.append(df)
    
v2_device_df = pd.concat(df_list).fillna('')
v2_device_df.sample(5)

In [ ]:
v3_device_df = []

with open(V3_DIR + 'devices.json') as fp:
    for line in fp:
        r = json.loads(line)
        del r['_id']
        v3_device_df.append(r)

v3_device_df = pd.DataFrame(v3_device_df).fillna('')
v3_device_df.sample(5)

In [ ]:
def get_netdisco(v):
    v = str(v)    
    if v.startswith('b64:'):
        return base64.b64decode(v[4:])
    return v

# oui_parser = common.OUIParser()

try:
    raw_device_df = pd.read_csv(TMP_DIR + 'combined_device_raw_table.csv')

except IOError:
    raw_device_df = pd.concat([v2_device_df, v3_device_df])
    raw_device_df['user_key'] = raw_device_df['user_key'].str.replace('-', '')
    raw_device_df['suspected_pc'] = raw_device_df['suspected_pc'].apply(lambda v: True if v == True else False)
    raw_device_df['user_agent_info'] = raw_device_df['ua_list'].apply(str)
    # raw_device_df['oui_friendly'] = raw_device_df['device_oui'].apply(lambda s: oui_parser.get_vendor(s))
    raw_device_df['oui_raw'] = raw_device_df['device_oui']
    raw_device_df['netdisco_info'] = raw_device_df['netdisco_device_info_list'].apply(get_netdisco)

    raw_device_df = raw_device_df[[
        'user_key', 'device_id', 'device_ip', 'suspected_pc',
        'device_vendor', 'device_name', 'device_type',   
        'user_agent_info', 'oui_friendly', 'oui_raw', 'dhcp_hostname', 'netdisco_info']]

    raw_device_df.set_index('user_key').to_csv(TMP_DIR + 'combined_device_raw_table.csv')

raw_device_df.sample(5)

In [ ]:
raw_device_df['device_id'].nunique()

## DNS

In [ ]:
device_domain_df = pd.read_csv(TMP_DIR + 'device_hostname_df.csv')
device_domain_df['domain'] = device_domain_df['remote_hostname'].apply(
    lambda s: tldextract.extract(str(s).replace('?', '')).registered_domain.lower()
)

del device_domain_df['remote_hostname']
device_domain_df = device_domain_df[device_domain_df['domain'] != '']
device_domain_df = device_domain_df.drop_duplicates()

# Get ad/trackers
blacklist_url = 'https://raw.githubusercontent.com/notracking/hosts-blocklists/master/domains.txt'
blacklisted_domain_set = set()
for token in requests.get(blacklist_url).text.split():
    components = token.split('/', 2)
    if len(components) == 3:
        blacklisted_domain_set.add(components[1])
        
# Remove ad/tracking
device_domain_df = device_domain_df[
    device_domain_df['domain'].apply(lambda s: s not in blacklisted_domain_set)
]

# Add user key
device_domain_df = pd.merge(
    device_domain_df,
    raw_device_df[['device_id', 'user_key']].drop_duplicates(),
    on='device_id',
    how='inner'
)

# Find domains contacted by at least 3 users
domain_count_df = device_domain_df.groupby('domain')['user_key'].nunique().to_frame('user_count').reset_index()
domain_count_df = domain_count_df[domain_count_df['user_count'] >= 3]
del domain_count_df['user_count']

device_domain_df = pd.merge(
    device_domain_df,
    domain_count_df,
    on='domain',
    how='inner'
)

device_domain_df.sample(20)

In [ ]:
device_domain_list_df = device_domain_df.groupby('device_id')['domain'].apply(lambda ss: '+'.join(set(ss))).to_frame('domains').reset_index()
device_domain_list_df.to_parquet(TMP_DIR + 'device_domain_list_no_ad_tracking.parquet')
device_domain_list_df.sample(10)

In [ ]:
ask_gpt_raw('which company operates "mystrom.ch"? output the company name only', max_tokens=20)

# Devices with names

## Sample

In [ ]:
named_device_df = raw_device_df[raw_device_df['device_name'] != ''] \
    [['device_id', 'suspected_pc', 'device_vendor', 'device_name', 'device_type']].copy()
print(len(named_device_df))
named_device_df.sample(10)

In [ ]:
sample_df = named_device_df.sample(300)
sample_df

In [ ]:
sample_df_copy = sample_df.copy()
del sample_df_copy['suspected_pc']
del sample_df_copy['device_id']
sample_df_copy.sample(10)

## GPT

In [ ]:
def contains(target_str, filter_list):
    target_str = target_str.lower()
    for s in filter_list:
        if s.lower() in target_str:
            return True
    return False

In [ ]:
def get_vendor(t):

    (_, r) = t
    vendor = r['device_vendor'].lower()
    
    if contains(vendor, ['ieee', 'espressif', 'hon hai']):
        device_label = f'{r["device_name"]} - {r["device_type"]}'
    else:
        device_label = f'{r["device_vendor"]} - {r["device_name"]} - {r["device_type"]}'

    prompt = f'I have an IoT device named "{device_label}". What is the company that makes this IoT device? Output the company\'s name only.',
    return ask_gpt_raw(prompt, max_tokens=20)


In [ ]:
ask_gpt_raw('I have an IoT device named "apple iphone". What type of IoT device is this? Output the name of the device type only.')

In [ ]:
def get_type(t):

    (_, r) = t
    device_label = f'{r["device_vendor"]} - {r["device_name"]} - {r["device_type"]}'

    prompt = f'I have an IoT device named "{device_label}". What type of IoT device is this? Output the name of the device type only.'
    return ask_gpt_raw(prompt, max_tokens=10)

In [ ]:
import multiprocessing.pool

with multiprocessing.pool.ThreadPool(processes=10) as pool:
    sample_df_copy['gpt_vendor'] = pool.map(get_vendor, sample_df_copy.iterrows())
    sample_df_copy['gpt_type'] = pool.map(get_type, sample_df_copy.iterrows())

sample_df_copy.sample(20)

# DHCP

## Sample

In [ ]:
dhcp_device_df = raw_device_df[raw_device_df['dhcp_hostname'] != ''] \
    [['dhcp_hostname', 'device_vendor', 'device_name', 'device_type']].copy()
print(len(dhcp_device_df))
dhcp_device_df = dhcp_device_df.sample(300)


In [ ]:
dhcp_sample_df = dhcp_device_df.copy()
dhcp_sample_df.sample(10)

## GPT

In [ ]:
get_dhcp_vendor = lambda s: ask_gpt_raw(f'I have an IoT device named "{s}". What is the company that makes this IoT device? Output the company\'s name only.', max_tokens=20)

get_dhcp_type = lambda s: ask_gpt_raw(f'I have an IoT device named "{s}". What type of IoT device is this? Output the name of the device type only.', max_tokens=20)



In [ ]:
print(get_dhcp_vendor('Suhas-iPhone'))
print(get_dhcp_type('Suhas-iPhone'))

In [ ]:
with multiprocessing.pool.ThreadPool(processes=10) as pool:
    dhcp_sample_df['gpt_dhcp_vendor'] = pool.map(get_dhcp_vendor, dhcp_sample_df['dhcp_hostname'])
    dhcp_sample_df['gpt_dhcp_type'] = pool.map(get_dhcp_type, dhcp_sample_df['dhcp_hostname'])

dhcp_sample_df.sample(20)

# Netdisco

## Sample

In [ ]:
netdisco_device_df = raw_device_df[
    (raw_device_df['netdisco_info'] != '[]') &
    (raw_device_df['netdisco_info'] != '') 
][['netdisco_info', 'device_vendor', 'device_name', 'device_type']].copy()
print(len(netdisco_device_df))
netdisco_device_df = netdisco_device_df.sample(300).reset_index(drop=True)


In [ ]:
netdisco_sample_df = netdisco_device_df.copy()
netdisco_sample_df.sample(10)

In [ ]:
df = netdisco_sample_df[netdisco_sample_df['netdisco_info'].apply(lambda s: '+' in str(s))]
df

In [ ]:
s = netdisco_sample_df.iloc[131]['netdisco_info'].decode('utf-8', 'replace')
print(s)
print(ask_gpt_raw('I have an IoT device with the following name. What is the company that makes this IoT device? Output the company\'s name only.\n\n' + s))
print(ask_gpt_raw('I have an IoT device with the following name. What type of IoT device is this? Output the name of the device type only.\n\n' + s))

## GPT

In [ ]:
get_netdisco_vendor = lambda s: ask_gpt_raw(f'I have an IoT device named "{s}". What is the company that makes this IoT device? Output the company\'s name only.', max_tokens=20)
get_netdisco_type = lambda s: ask_gpt_raw(f'I have an IoT device named "{s}". What type of IoT device is this? Output the name of the device type only.', max_tokens=20)

In [ ]:
with multiprocessing.pool.ThreadPool(processes=10) as pool:
    netdisco_sample_df['gpt_netdisco_vendor'] = pool.map(get_netdisco_vendor, netdisco_sample_df['netdisco_info'])
    netdisco_sample_df['gpt_netdisco_type'] = pool.map(get_netdisco_type, netdisco_sample_df['netdisco_info'])

netdisco_sample_df.sample(50)

# DNS

Doesn't work well it seems

## Sample

In [ ]:
dns_device_df = raw_device_df[raw_device_df['device_name'] != ''] \
    [['device_id', 'device_vendor', 'device_name', 'device_type']]

dns_device_df = pd.merge(dns_device_df, device_domain_df, on='device_id', how='inner').sample(300)

dns_device_df.sample(50)

# Combining multiple features

... with at least two features available

## Combine

In [ ]:
combined_df = raw_device_df.fillna('').copy()

combined_df = pd.merge(combined_df, device_domain_list_df, on='device_id', how='left').fillna('')

combined_df['has_user_label'] = combined_df['device_name'].apply(lambda s: 1 if s != '' else 0)

combined_df['has_dhcp'] = combined_df['dhcp_hostname'].apply(lambda s: 1 if s != '' else 0)

combined_df['has_netdisco'] = combined_df['netdisco_info'].apply(lambda s: 1 if s != '[]' and s != '' else 0)

combined_df['has_domains'] = combined_df['domains'].apply(lambda s: 1 if s != '' else 0)

combined_df['feature_count'] = combined_df['has_user_label'] + combined_df['has_dhcp'] + combined_df['has_netdisco'] + combined_df['has_domains']

print(len(combined_df))
print(combined_df['device_id'].nunique())

combined_df.sample(10)

In [ ]:
df = combined_df.value_counts('feature_count', dropna=False).to_frame('device_count')
df['percent'] = (df['device_count'] * 100.0 / df['device_count'].sum()).round(1)
df

## Sample

In [ ]:
two_feature_raw_df = combined_df[combined_df['feature_count'] >= 2]
print(len(two_feature_raw_df))
two_feature_raw_df.head(2)

### Check the length of fields

In [ ]:
field_length_df = two_feature_raw_df.copy()

interesting_fields = ['dhcp_hostname', 'netdisco_info', 'device_vendor', 'device_name', 'device_type']

for col in interesting_fields:
    field_length_df['length_' + col] = field_length_df[col].str.len()
    
for col in interesting_fields:
    print(f'Max of {col}: ' + str(field_length_df['length_' + col].max()))

In [ ]:
field_length_df[['device_name', 'length_device_name']].sort_values(by='length_device_name', ascending=False).head()

In [ ]:
df = field_length_df[['netdisco_info', 'length_netdisco_info']].sort_values(by='length_netdisco_info', ascending=False)
df.head()

In [ ]:
len(df[df['length_netdisco_info'] > 1000])

### Truncating long fields

In [ ]:
len(two_feature_raw_df)

In [ ]:
# two_feature_df = two_feature_raw_df.copy().sample(int(len(two_feature_raw_df) / 2), random_state=0)
two_feature_df = two_feature_raw_df.copy()

for col in interesting_fields:
    two_feature_df[col] = two_feature_df[col].apply(lambda s: str(s)[0:1200])
    
two_feature_df.sample(2)

## GPT

In [ ]:
def infer_name(pbar_inst, df_row):
    
    (_, row) = df_row
    output = {}

    with T_LOCK:
        pbar_inst.update(1)    
    
    if row['device_name'] != '':
        output['gpt_user_vendor'] = get_vendor(df_row)
        output['gpt_user_type'] = get_type(df_row)

    if row['dhcp_hostname'] != '':
        output['gpt_dhcp_vendor'] = get_dhcp_vendor(row['dhcp_hostname'])
        output['gpt_dhcp_type'] = get_dhcp_type(row['dhcp_hostname'])    

    if row['netdisco_info'] != '':
        output['gpt_netdisco_vendor'] = get_netdisco_vendor(row['netdisco_info'])
        output['gpt_netdisco_type'] = get_netdisco_type(row['netdisco_info'])        

    return output

In [ ]:
with tqdm.tqdm(total=len(two_feature_df), smoothing=0.1) as pbar:
    with multiprocessing.pool.ThreadPool(processes=15) as pool:
        gpt_list = pool.map(lambda df_row: infer_name(pbar, df_row), two_feature_df.iterrows())

## Remove bad answers

In [ ]:
gpt_two_feature_df = []
for (ix, gpt_dict) in enumerate(gpt_list):
    if gpt_dict is None:
        continue
    df_row_dict = two_feature_df.iloc[ix].to_dict()
    df_row_dict.update(gpt_dict)
    gpt_two_feature_df.append(df_row_dict)

gpt_two_feature_df = pd.DataFrame(gpt_two_feature_df).fillna('')

# Filter out crappy answers

def remove_bad_answer(s):
    if '___' in s:
        return ''
    
    if s.endswith('.'):
        return s[0:-1]
    
    if 'unknown' in s.lower() or 'espressif' in s.lower() or 'ESP' in s:
        return ''
    
    if s == 'DLNA_DMR':
        return 'Streaming Device'
    
    return s

for col in gpt_two_feature_df.columns:
    if col.startswith('gpt_'):
        gpt_two_feature_df[col] = gpt_two_feature_df[col].apply(remove_bad_answer) 

print(len(gpt_two_feature_df))
print(gpt_two_feature_df['device_id'].nunique())
gpt_two_feature_df.sample(10)

In [ ]:
gpt_two_feature_df.iloc[0]

In [ ]:
# Remove certain columns for student to build classifier
df = gpt_two_feature_df[
    ['device_id', 'oui_raw', 'domains'] +
    [col for col in gpt_two_feature_df.columns if col.startswith('gpt_')]
]

df.to_parquet('../tmp-data/gpt_two_feature_df_for_ML.parquet')

df.sample(5)

## Check vendor consistency

### Cross product consistency checks

In [ ]:
stop_word_set = {
      'electronics', 'smart', 'inc', 'technology', 'espressif', 'assistant', 'technologies', 'corporation'
}

equivalent_companies = [
    {'lg', 'lg electronics'},
    {'nest', 'google'},
    {'myq', 'chamberlain'},
    {'xiaomi', 'yeelight'}
]

def is_same_vendor(v1, v2):
    
    v1 = v1.lower().replace('.', '').replace('-', '')
    v2 = v2.lower().replace('.', '').replace('-', '')
    
    if v1 == '' or v2 == '':
        return ''
    
    # Same vendor if one is a substring of another
    if len(v1) >= 3 and v1 in v2 and v1 not in stop_word_set:
        return 'substring:' + v1
    if len(v2) >= 3 and v2 in v1 and v2 not in stop_word_set:
        return 'substring:' + v2
    
    # Same vendor if sharing at least one token that is not a stop word
    v1_tokens = set(v1.split())
    v2_tokens = set(v2.split())
    common_tokens = (v1_tokens & v2_tokens) - stop_word_set
    if common_tokens:
        return 'common_tokens:' + '+'.join(common_tokens)
    
    # Some of these tokens are substrings of each other
    for (t1, t2) in itertools.product(v1_tokens, v2_tokens):        
        if len(t1) >= 3 and t1 in t2 and t1 not in stop_word_set:
            return 'common_token_substring:' + t1
        if len(t2) >= 3 and t2 in t1 and t2 not in stop_word_set:
            return 'common_token_substring:' + t2
    
    return 'different'
    

In [ ]:
columns_to_check = ['gpt_user_vendor', 'gpt_dhcp_vendor', 'gpt_netdisco_vendor', 'oui_friendly', 'domains_friendly']

In [ ]:
x = {1,2,3}
x.pop()
x

In [ ]:
gpt_two_feature_consistency_df = gpt_two_feature_df.copy()

gpt_two_feature_consistency_df['domains_friendly'] = gpt_two_feature_consistency_df['domains'].apply(
    lambda s: ' '.join(set([tldextract.extract(reg_domain).domain for reg_domain in s.split('+')]))
)

# Check across the columns for consistency
for (col1, col2) in itertools.combinations(columns_to_check, 2):
    new_col = f'consistency:{col1}:{col2}'
    gpt_two_feature_consistency_df[new_col] = gpt_two_feature_consistency_df.apply(
        lambda r: is_same_vendor(r[col1], r[col2]), axis=1
    )
    
# Consolidate
def consolidate_consistency(r):
    r = r.to_dict()
    
    common_term_set = set()
    for (k, v) in r.items():
        if k.startswith('consistency:') and ':' in v:
            common_term_set.add(v.split(':', 1)[1])
    
    # Some of the common terms are substrings of each other; merge these terms
    terms_to_remove = set()
    for (t1, t2) in itertools.permutations(common_term_set, 2):
        if len(t1) >=3 and t1 in t2:
            terms_to_remove.add(t2)
    
    common_term_set -= terms_to_remove
    
    # Merge equiv companies
    if common_term_set in equivalent_companies:
        common_term_set.pop()
        
    # If there are multiple terms and one of them is X (e.g. Spotify), remove X.
    if len(common_term_set) > 1:
        common_term_set -= {'spotify', 'google'}
        
    return '+'.join(common_term_set)

gpt_two_feature_consistency_df['consolidated_vendor'] = gpt_two_feature_consistency_df.apply(lambda r: consolidate_consistency(r), axis=1)

# Remove stop words and irrelevant results
def clean_consolidated_vendor(s):

    if 'hewlett' in s or 'packard' in s:
        return 'hp'
    
    if 'raspberry' in s:
        return 'raspberry-pi'
    
    if 'wemo' in s:
        return 'belkin'
    
    if s in ('ind', 'one', 'hon', 'shenzhen', 'electric', 'media', 'hom', 'ltd', 'digital', 'things', 'the', 'night', 'security'):
        return ''
    
    if s == 'free':
        return 'freebox'

    s = s.replace('electronics', '') \
        .replace('international inc', '') \
        .replace('inc', '') \
        .replace('corporation', '') \
        .replace('networks', '') \
        .replace('labs', '') \
        .replace('group', '') \
        .replace('llc', '') \
        .replace('foundation', '') \
        .replace('technology', '') \
        .replace('technologies', '') 
    
    return s.strip()
    
gpt_two_feature_consistency_df['consolidated_vendor'] = gpt_two_feature_consistency_df['consolidated_vendor'].apply(clean_consolidated_vendor)

In [ ]:
gpt_two_feature_consistency_df.iloc[4683].to_dict()

In [ ]:
gpt_two_feature_consistency_df.sample(5)

### Clean up

In [ ]:
# Find all the unambiguous cases
df = gpt_two_feature_consistency_df.groupby('consolidated_vendor')['device_id'].nunique().sort_values(ascending=False).to_frame('device_count')
df = df[df['device_count'] >= 2].reset_index()
df = df[~df['consolidated_vendor'].str.contains('+', regex=False)]
df = df[df['consolidated_vendor'] != '']
print(df.to_string())



In [ ]:
gpt_two_feature_clean_df = pd.merge(
    gpt_two_feature_consistency_df,
    df[['consolidated_vendor']],
    on='consolidated_vendor',
    how='inner'
)

In [ ]:
print(combined_df['device_id'].nunique())
print(two_feature_raw_df['device_id'].nunique())
print(two_feature_df['device_id'].nunique())
print(gpt_two_feature_clean_df['device_id'].nunique())

In [ ]:
gpt_two_feature_clean_df[gpt_two_feature_clean_df['consolidated_vendor'] == 'hom']

### Output

In [ ]:
# def get_clean_vendor(r):
#     if r['gpt_user_vendor']:
#         return r['gpt_user_vendor']
#     if r['gpt_netdisco_vendor']:
#         return r['gpt_netdisco_vendor']
#     if r['gpt_dhcp_vendor']:
#         return r['gpt_dhcp_vendor']
#     return ''

def get_clean_type(r):
    if r['gpt_user_type']:
        return r['gpt_user_type']
    if r['gpt_netdisco_type']:
        return r['gpt_netdisco_type']
    if r['gpt_dhcp_type']:
        return r['gpt_dhcp_type']
    return ''

In [ ]:
gpt_clean_df = gpt_two_feature_clean_df.copy()
gpt_clean_df['gpt_clean_vendor'] = gpt_clean_df['consolidated_vendor']
gpt_clean_df['gpt_clean_type'] = gpt_clean_df.apply(get_clean_type, axis=1)

gpt_clean_df = gpt_clean_df[
    (gpt_clean_df['gpt_clean_vendor'] != '') &
    (gpt_clean_df['gpt_clean_type'] != '')
]

gpt_clean_output_df = gpt_clean_df[['user_key', 'device_id', 'gpt_clean_vendor', 'gpt_clean_type']]
gpt_clean_output_df.to_parquet('../tmp-data/gpt_clean_device_ident.parquet')

print(gpt_clean_output_df['device_id'].nunique())
gpt_clean_output_df.sample(10)

### Validation



#### Notes

#### Analysis

In [ ]:
fields_to_check = ['oui_friendly', 'dhcp_hostname', 'netdisco_info', 'domains', 'device_vendor', 'device_name', 'device_type']

In [ ]:
gpt_clean_df.sample(2)

In [ ]:
validation_result_df = []

for (_, row) in gpt_clean_df.iterrows():
    
    vendor = row['gpt_clean_vendor']
    
    vendor = vendor.replace(' home', '').replace('-pi', '')
    
    device_id = row['device_id']
    
    match_count = 0
    output_record = {
        'device_id': device_id
    }
        
    for field in fields_to_check:
        value = str(row[field]).lower().replace('-', '')
        if vendor in value:
            match_count += 1
            output_record['matched_' + field] = 1
        else:
            output_record['matched_' + field] = 0
            
    output_record['total_matches'] = match_count
    validation_result_df.append(output_record)
    
validation_result_df = pd.DataFrame(validation_result_df)
validation_result_df = pd.merge(
    validation_result_df, gpt_clean_df,
    how='inner', on='device_id'
)

validation_result_df.sample(5)

In [ ]:
viz_df = validation_result_df.groupby('total_matches')['device_id'].nunique().to_frame('device_count')
viz_df['percent'] = (viz_df['device_count'] * 100.0 / viz_df['device_count'].sum()).round(1)
viz_df

In [ ]:
df = validation_result_df[validation_result_df['total_matches'] == 0].reset_index(drop=True)
df.sample(10)